In [46]:
import sys
sys.path.append('..')

from pub_crawler.tests.spiders.test_beer_spider import *

In [47]:
response = get_response(ARDENT_BREWING_MENU_DATA)

In [48]:
from bs4 import BeautifulSoup
import re
soup = BeautifulSoup(response.text, 'html.parser')

In [65]:
soup.find(text=re.compile("ABV"))

'5% ABV'

In [50]:
import scrapy

In [51]:
request = scrapy.Request("https://ardentcraftales.com/menus")

In [116]:
def correct_elements_exist(div):
    # Only one element per search
    name = div.find_all(class_=re.compile("name"))
    style = div.find_all(class_=re.compile("beer-style"))
    abv = div.find_all(class_=re.compile("abv"))
    only_one_element = (0 < len(elements) < 5 for elements in [abv, name, style])
    return all(only_one_element)
    # All required elements exist
    

def data_exists(div):
    name = div.find(class_=re.compile("name")).text
    style = div.find(class_=re.compile("beer-style")).text
    abv = div.find(class_=re.compile("abv")).text
    return all(name, style, abv)

In [124]:
"""
The basic idea here works if we can supplement the names for classes with different spellings AND
supplement for finding text matches as well

Some issues found here:
- The name includes the style as bs4 is printing all text contained in an element
- The search feels slow and could probably be optimized with better filtering technique
- This works with Ardent, but need to test on other sites as well before starting to commit code
- The `correct_elements_exist` filter func makes a HUGE assumption of have less that 5 elements match
"""

candidates = soup.find_all(correct_elements_exist)

data = []
for candidate in candidates:
    d = {
        "name": candidate.find(class_=re.compile("name")).text.strip(),
        "style": candidate.find(class_=re.compile("beer-style")).text.strip(),
        "abv": candidate.find(class_=re.compile("abv")).text.strip()
    }
    if d not in data:
        data.append(d)
        
data

[{'name': 'Altbier\n        \nAltbier', 'style': 'Altbier', 'abv': '5% ABV'},
 {'name': 'Highpoint DIPA\n        \nIPA - Imperial / Double',
  'style': 'IPA - Imperial / Double',
  'abv': '8% ABV'},
 {'name': 'Pomegranate Gose With Ginger & Mint\n        \nSour - Fruited Gose',
  'style': 'Sour - Fruited Gose',
  'abv': '4.5% ABV'},
 {'name': 'Helles Lager\n        \nLager - Helles',
  'style': 'Lager - Helles',
  'abv': '4.5% ABV'},
 {'name': 'Double Dry Hopped IPA\n        \nIPA - American',
  'style': 'IPA - American',
  'abv': '6.8% ABV'},
 {'name': 'Spiced Cranberry Gose\n        \nSour - Fruited Gose',
  'style': 'Sour - Fruited Gose',
  'abv': '4.5% ABV'},
 {'name': 'Stronger Than Hate\n        \nStout - Imperial / Double Milk',
  'style': 'Stout - Imperial / Double Milk',
  'abv': '10% ABV'},
 {'name': 'ESB\n        \nExtra Special / Strong Bitter',
  'style': 'Extra Special / Strong Bitter',
  'abv': '5.5% ABV'},
 {'name': 'Summit DIPA\n        \nIPA - Imperial / Double New En